# Lab 5a: HashCash  
En este lab, vamos a hablar sobre una aplicación de Hashing que se puede usar para combatir el spam. De hecho, no sólo es útil para spam. La función que vamos a ver, llamada HashCash, sirve como base para el algoritmo de consenso que se utiliza en BitCoin, un método que comúnmente se denomina como Proof-of-Work, ya que requiere cierta cantidad de "trabajo" promedio para calcular. Vamos a implementar esta función y analizar un poco el trabajo que se requiere para calcularla.  
  
Para empezar, lee las primeras 3 secciones de este documento: http://www.hashcash.org/papers/hashcash.pdf  
  
En el documento mencionan que el string a calcular normalmente se agrega a un string basado en el servicio que se está usando. También se le agregan otros detalles, como la versión de la función, el número de bits a chequear, un timestamp, y algún denominador del servicio/usuario. Debajo hay algunos ejemplos de cómo se ve un formato de string de HashCash en la vida real. Vamos a usar SHA-1 como el Hash.  
  
Ejemplo de formato de HashCash:

In [1]:
from cryptography.hazmat.primitives import hashes
import time
import random

def hash(msg: str):
	digest = hashes.Hash(hashes.SHA1())
	digest.update(msg.encode("unicode_escape"))
	return digest.finalize().hex()

example = "1:20:1303030600:jpazos@utec.edu.pe::McMybZIhxKXu57jd:ckvi"

hash(example)

'0ec57d01555bb613eeb6a3c20a0fcc64ef581794'

## Ejemplo de verificación correcta de hash. Se tienen 24 0-bits según el string, y el hash tiene 6 0s en hex.

In [10]:
hash("1:24:040806:foo::511801694b4cd6b0:1e7297a")

'0000008e3caaff34c595a55f7cc53c6b1cd385d1'

# Ejercicio: implementa la versión recursiva de HashCash.  
Puedes verificar que funciona calculando el hash y verificando que tiene la cantidad
de bits deseada. Nota que estamos trabajando con Hex por ahora, entonces pueden haber más bits de 0 de los necesarios, pero con tal que
hayan por lo menos 8 (o los que especifiques) sería correcto.

In [4]:
def getBaseString(*args):
	result = ""
	for s in args:
		result += str(s) + ":"
	return result[:-1]

def recursiveHashCash(ver: int, bits: int, resource: str, iv: str, ctr: int):
	curTime = str(int(time.time()))
	baseStr = str(ver) + ':' + str(bits) + ':' + curTime + ':' + resource + ':' + iv + ':' + str(ctr);
	hashed = hash(baseStr)

	if hashed[:bits] == "0"*bits:
		return hashed
	else:
		return recursiveHashCash(ver, bits, resource, iv, ctr)

def checkBits(baseStr: str, bits: int):
	hashed = hash(baseStr)
	bitCheck = bits//4
	return hashed[:bitCheck] == "0"*bitCheck


In [ ]:
version = 1
bits = 2
resource = "UTECSecurity"

res = recursiveHashCash(version,bits,resource,str(random.randint(1e17,1e18)),0)
print(res)
print(hash(res))

## Ejercicio: Trata aumentando la cantidad de bits, ¿qué pasa? Lamentablemente python nos empieza a fallar. ¿Qué pasa si tratas de agregar tail recursion?

In [3]:
from tail_recursion import tail_recursive, recurse

@tail_recursive
def tailRecursiveHashCash(ver: int, bits: int, resource: str, iv: str, ctr: int):
    curTime = str(int(time.time()))
    baseStr = str(ver) + ':' + str(bits) + ':' + curTime + ':' + resource + ':' + iv + ':' + str(ctr)
    hashed = hash(baseStr)

    if hashed[:bits] == "0"*bits:
        return hashed
    recurse(ver, bits, resource, iv, ctr)

In [ ]:
version = 1
bits = 2
resource = "UTECSecurity"

res = tailRecursiveHashCash(version,bits,resource,str(random.randint(1e17,1e18)),0)
print(res)
print(hash(res))

## Ejercicio: Para librarnos del problema anterior, podemos usar iteration. Implementa una versión iterativa de HashCash.

In [ ]:
def iterativeHashCash(ver: int, bits: int, resource: str, iv: str, ctr: int):
	hashed = ''
	while True:
		curTime = int(time.time())
		baseStr = getBaseString(ver, bits, curTime, resource, iv, ctr)
		hashed = hash(baseStr)
		if hashed[:bits] == "0"*bits:
			break
	return hashed

version = 1
bits = 4
resource = "UTECSecurity"

res = iterativeHashCash(version,bits,resource,str(random.randint(1e17,1e18)),0)
print(res)
print(hash(res))

## Ejercicio: Ahora que podemos correr la función con más bits, probemos el performance conforme incrementemos los bits. 

In [ ]:
for i in range(4,28,4):
	print("="*10,"\n",i,"bits:")
	%time _

Lamentablemente, trabajar con hex nos limita bastante, ya que solo podemos incrementar la cantidad de bits de a 4.  
Queremos poder incrementar de a 1 para poder calcular mejor el cambio en tiempos mientras incrementamos los bits.  
  
# Ejercicio: Implementar de iterativeHashCashPartial y CheckBitsPartial, que permite analizar el ByteArray, de modo que podamos incrementar bit por bit la dificultad de HashCash.

In [ ]:
def hextobin(h):
  return bin(int(h, 16))[2:].zfill(len(h) * 4)

def iterativeHashCashPartial(ver: int, bits: int, resource: str, iv: str, ctr: int):
	baseStr = _

	while not _
		_
	
	return _

def checkBitsPartial(baseStr: str, bits: int):
	
	return _

res = iterativeHashCashPartial(version,6,resource,str(random.randint(1e17,1e18)),0)
print(res)
print(hash(res))
print(hextobin(hash(res)))

# Calcular tiempo para diferentes 0-bit-strings.

In [ ]:
for i in range(24):
	print("="*10,"\n",i,"bits:")
	%time _

# Ejercicio: Dibuja un gráfico de los tiempos con distintas cantidades de bits a chekear. ¿Se ve como esperarías? ¿Por qué hay diferencias?
Puedes usar librerias como matplotlib o seaborn

# Ejercicio: Una de las aplicaciones que se tenía en mente originalmente para HashCash era combatir el Spam en email. Describe cómo se podría utilizar HashCash para ayudar con este problema.

# Bonus:  Bitcoin, una de las aplicaciones que hace uso de HashCash usa la función SHA-256 para hacer hashing. También se usan IVs y contadores más grandes (alrededor de 64 bytes cada uno) para evitar la probabilidad de reuso de un IV. Extiende la longitud de los IVs y el CTR en tus cálculos y compara las diferencia de tiempo. ¿Es significativa?

# Bonus: Ejercicio: Bitcoin mantiene una dificultad de hashing de modo que un bloque sea agregado al Blockchain mas o menos cada 10 minutos. Usando la extensión del problema anterior, y asumiendo una escala teórica del incremento del tiempo, ¿más o menos qué longitud debería tener el string de 0s para que demore 10 minutos encontrar una solución según tus tiempos?